In [2]:
!pip3 install pandas
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as data
import torch.nn.init as init

from lib.layers import *
from lib.utils.timer import Timer
from lib.utils.data_augment import preproc
from lib.modeling.model_builder import create_model
from lib.modeling.discriminator.DNet import D_Net
from lib.dataset.dataset_factory import load_data
from lib.utils.config_parse_mimic_semi_GAN_trained_frozen_CONF_LOC_VOC_COCO import cfg
from lib.utils.config_parse_mimic_VOC_COCO_ZERO import cfg1
# from lib.utils.config_parse_mimic_VOC_COCO_LR_ARD import cfg
from lib.utils.eval_utils import *
from lib.utils.visualize_utils import *

In [7]:
test_loader = load_data(cfg.DATASET, 'test') if 'test' in cfg.PHASE1 else None

In [8]:
model_student, priorbox = create_model(cfg.MODEL)
model_student.loc.load_state_dict(torch.load(cfg.LOG_DIR+'loc.pth'))
model_student.conf.load_state_dict(torch.load(cfg.LOG_DIR+'conf.pth'))
# model_student.load_state_dict(
#     torch.load('./Weights/DarkNet53_Residual_Blocks_Ones_VOC_COCO/ssd_lite_mobilenet_v2_voc_epoch_249_MAX.pth'))
# priors = Variable(priorbox.forward(), volatile=True)
# detector = Detect(cfg.POST_PROCESS, priors.cpu())

==>Feature map size:
[(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]


In [14]:
model_student.conf[0].weight

Parameter containing:
tensor([[[[ 1.3185e-02, -4.3213e-03,  1.4823e-02],
          [ 3.2780e-02, -2.5385e-02,  6.8572e-03],
          [ 1.0549e-02, -3.7347e-02, -1.4727e-02]],

         [[ 7.9917e-03, -5.9146e-03,  1.5076e-02],
          [ 1.9999e-02, -3.2863e-02, -2.0859e-03],
          [ 1.1350e-02, -3.2956e-02, -7.8733e-03]],

         [[-2.5234e-02, -2.0167e-02, -9.9620e-03],
          [-1.1213e-02, -2.9266e-02, -1.5218e-02],
          [-2.6531e-02, -3.3449e-02, -2.4215e-02]]],


        [[[-6.3694e-02, -2.1300e-02,  2.1416e-02],
          [ 1.3037e-01,  3.7967e-01,  4.1983e-02],
          [-1.8793e-01, -2.7921e-01, -3.8335e-02]],

         [[-6.7306e-02,  5.3145e-02,  1.8353e-03],
          [ 2.6662e-01,  7.4863e-01,  1.0166e-01],
          [-3.3641e-01, -6.0844e-01, -7.6828e-02]],

         [[-1.5798e-02, -1.0921e-02,  2.6319e-02],
          [ 4.2062e-02,  1.7312e-01,  1.5455e-03],
          [-6.2128e-02, -1.5648e-01,  1.0293e-03]]],


        [[[-1.0511e-01,  9.7661e-02, -7.8915

In [16]:
model_teacher = create_model(cfg.MODEL_MIMIC)
model_teacher.load_state_dict(torch.load('./darknet53_ssd_voc_77.3.pth'))

==>Feature map size:
[(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]


<All keys matched successfully>

Parameter containing:
tensor([[[[ 1.4947e-02,  1.4249e-04,  1.8321e-02],
          [ 3.1701e-02, -2.0935e-02,  9.0623e-03],
          [ 1.0202e-02, -3.1796e-02, -1.1235e-02]],

         [[ 1.0202e-02, -1.0932e-03,  1.8799e-02],
          [ 2.0417e-02, -2.7206e-02,  1.4878e-03],
          [ 1.1425e-02, -2.7299e-02, -4.5147e-03]],

         [[-2.1829e-02, -1.6314e-02, -6.4126e-03],
          [-9.8078e-03, -2.6484e-02, -1.2967e-02],
          [-2.4728e-02, -3.0059e-02, -2.1704e-02]]],


        [[[-6.4440e-02,  4.4501e-02,  6.5879e-02],
          [ 9.3905e-02,  2.6670e-01,  5.1718e-02],
          [-1.4706e-01, -1.5188e-01, -6.0276e-02]],

         [[-1.0183e-01,  1.3220e-01,  1.8830e-02],
          [ 2.4074e-01,  6.5986e-01,  8.9881e-02],
          [-3.3628e-01, -4.8546e-01, -1.1940e-01]],

         [[-5.3875e-02,  1.4022e-01, -4.2159e-03],
          [ 1.3528e-01,  3.2877e-01,  1.1022e-01],
          [-2.0900e-01, -2.2832e-01, -9.1857e-02]]],


        [[[-9.6115e-02,  1.2291e-01, -4.0348

In [ ]:
weights = torch.load('./Weights/DarkNet53_Original/ssd_darknet_53_voc_epoch_99_MAX_74.7.pth')

In [ ]:
conf = {i[5:]:weights[i] for i in weights if 'conf' in i}

In [ ]:
conf

In [ ]:
torch.save(conf, 'conf.pth')

In [ ]:
trainable_scope = 'base,norm,extras,transform0,transform1'
untrainable_scope = 'loc,conf'
for module in trainable_scope.split(','):
    if hasattr(model_student, module):
        # print(getattr(self.model, module))
        for param in getattr(model_student, module).parameters():
            param.requires_grad = True
for module in untrainable_scope.split(','):
    if hasattr(model_student, module):
        # print(getattr(self.model, module))
        for param in getattr(model_student, module).parameters():
            param.requires_grad = False

In [ ]:
model_student.base[0][0].weight.requires_grad

In [ ]:
model_student.loc[0].weight.requires_grad

In [ ]:
model_original, priorbox = create_model(cfg1.MODEL)
model_original.load_state_dict(
    torch.load('./Weights/DarkNet53_Medium_Ones_VOC_COCO_ZERO/ssd_lite_mobilenet_v2_voc_epoch_300_MAX_72.3.pth'))

In [ ]:
model_student.eval().cpu()
model_teacher.eval().cpu()
model_original.eval().cpu()
print('Sucsessful')

In [ ]:
dataset = test_loader.dataset
num_images = len(dataset)
num_images

In [ ]:
img = dataset.pull_image(1)
rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
scale = torch.tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]], dtype=torch.float)
plt.imshow(img)

In [ ]:
images = Variable(dataset.preproc(img)[0].unsqueeze(0), volatile=True)
out_student, FM_student = model_student(images, phase='train_mimic')
FM_teacher = model_teacher(images, phase='feature')
out_original, FM_original = model_original(images, phase='train_mimic')

In [ ]:
eval_student= model_student(images, phase='eval')
eval_teacher= model_teacher(images, phase='eval')
eval_original= model_original(images, phase='eval')

In [ ]:
plt.figure(figsize=(10,10))
colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
plt.imshow(rgb_image)  # plot the image for matplotlib
currentAxis = plt.gca()
detections = detector.forward(eval_student)
print(detections.size())
# labels = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',\
#           'boat', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'bottle', 'chair',\
#           'couch', 'potted plant', 'dining table', 'tv']
labels = (  # always index 0
    'person', 'bicycle', 'car', 'motorbike',
    'aeroplane', 'bus', 'train', 'boat', 'bird',
    'cat', 'dog', 'horse', 'sheep',
    'cow', 'bottle', 'chair',
    'sofa', 'pottedplant', 'diningtable', 'tvmonitor')

for i in range(detections.size(1)):
    j = 0
    while detections[0,i,j,0] >= 0.3:
        score = detections[0,i,j,0]
        label_name = labels[i-1]
        display_txt = '%s: %.2f'%(label_name, score)
        pt = (detections[0,i,j,1:]*scale).cpu().numpy()
        coords = (pt[0], pt[1]), pt[2]-pt[0]+1, pt[3]-pt[1]+1
#         print(label_name)
#         print(score)
        color = colors[i]
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
        currentAxis.text(pt[0], pt[1], display_txt, bbox={'facecolor':color, 'alpha':0.5})
        j+=1

In [ ]:
print(FM_student[0].size())
print(FM_teacher[0].size())
print(FM_original[0].size())

In [ ]:
# S1 = FM_student[0].permute(0,2,3,1).view(-1,512)
# print(S1.size())
# T1 = FM_teacher[0].permute(0,2,3,1).view(-1,512)
# print(T1.size())
# O1 = FM_original[0].permute(0,2,3,1).view(-1,512)
# print(O1.size())
S1 = FM_student[0].view(512,-1)
print(S1.size())
T1 = FM_teacher[0].view(512,-1)
print(T1.size())
O1 = FM_original[0].view(512,-1)
print(O1.size())

In [ ]:
dfs = pd.DataFrame(S1.transpose(0,1).detach().numpy())
dft = pd.DataFrame(T1.transpose(0,1).detach().numpy())
dfo = pd.DataFrame(O1.transpose(0,1).detach().numpy())

In [ ]:
print(dfs.shape)
dfs.head()

In [ ]:
st = dfs.corrwith(dft)
st

In [ ]:
st = dfs.corrwith(dft)
ot = dfo.corrwith(dft)
so = dfs.corrwith(dfo)
print(st.mean())
print(ot.mean())
print(so.mean())

In [ ]:
model_student.eval().cuda()
model_teacher.eval().cuda()
model_original.eval().cuda()
mean_corr = [[0,0,0] for i in range(6)]
for i in range(100):
    img = dataset.pull_image(i)
    images = Variable(dataset.preproc(img)[0].unsqueeze(0).cuda(), volatile=True)
    out_student, FM_student = model_student(images, phase='train_mimic')
    FM_teacher = model_teacher(images, phase='feature')
    out_original, FM_original = model_original(images, phase='train_mimic')
    for j in range(6):
#         chanal = FM_student[j].size(1)
#         S = FM_student[j].permute(0,2,3,1).view(-1,chanal)
#         T = FM_teacher[j].permute(0,2,3,1).view(-1,chanal)
#         O = FM_original[j].permute(0,2,3,1).view(-1,chanal)

        S = FM_student[j].view(FM_student[j].size(1),-1)
        T = FM_teacher[j].view(FM_teacher[j].size(1),-1)
        O = FM_original[j].view(FM_original[j].size(1),-1)
        dfs = pd.DataFrame(S.transpose(0,1).cpu().detach().numpy())
        dft = pd.DataFrame(T.transpose(0,1).cpu().detach().numpy())
        dfo = pd.DataFrame(O.transpose(0,1).cpu().detach().numpy())
        st = dfs.corrwith(dft)
        ot = dfo.corrwith(dft)
        so = dfs.corrwith(dfo)
        mean_corr[j][0] += st.mean()
        mean_corr[j][1] += ot.mean()
        mean_corr[j][2] += so.mean()
    if i%20==0 and i>0:
        print(mean_corr)
        print('====================')

In [ ]:
f = open('./Test/ResNet-channel.txt', 'w')
print('Stud-Teach||Orig-Teach||Orig-Stud')
f.write('Stud-Teach||Orig-Teach||Orig-Stud \n')
for i in range(6):
    print('{st:9.5f} || {ot:8.5f} || {so:8.5f}'.format(st = mean_corr[i][0]/100,\
                                                     ot=mean_corr[i][1]/100,so=mean_corr[i][2]/100))
    f.write('{st:9.5f} || {ot:8.5f} || {so:8.5f} \n'.format(st = mean_corr[i][0]/100,\
                                                     ot=mean_corr[i][1]/100,so=mean_corr[i][2]/100))
f.close()